University of Zagreb<br>
Faculty of Electrical Engineering and Computing

# Text Analysis and Retrieval (TAR)

<a href="http://www.fer.unizg.hr/predmet/apt">http://www.fer.unizg.hr/predmet/apt</a>

2015./2016.

# Project theme 13: Tweet classification


(c) 2016 Group nedovrs: Tomislav Marinković, Josip Milić, Domagoj Pereglin

*Version 0.95*

Date: **05.06.2016.**<br>

<h1>Tweet preprocessor</h1>

<h3>Packages:</h3>

In [2]:
# -*- coding: utf-8 -*-
from subprocess import call, Popen
import os
import codecs
import re, string

<h3>Tweet text cleaner:</h3>

In [10]:
def purge(str):
    purged =re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str).strip()
    purged_spl = purged.split(' ')
    if (purged) and (purged_spl[-1][0] in '^'):
        purged = ' '.join(purged_spl[:-1]).strip()
    for forbidden in '!&.?-–^,;:"\'/[]{}()<>':
        while forbidden in purged:
            purged = purged.replace(forbidden,'')
    
    purged = purged.strip()
    return purged

def get_tweet( annotation_test ):
    dataset_dir_path = '../datasets'
    dataset_test,tweet_id_test,tweet_category_test = annotation_test.split(';')
    tweet_category_test = int(tweet_category_test)
    dataset_file_test = open(dataset_dir_path + '\\' + dataset_test + '_tweets.txt','r')
    for line in dataset_file_test:
        if tweet_id_test == line.split('|',2)[0]:
            pur = purge(line.rstrip().split('|',2)[2])
            if pur == '':
                return 'poveznica'
            return pur
    dataset_file_test.close()

<h3>Annotated dataset preprocessing:</h3>

In [5]:
import string
def important ( str ):
    response = True
    for word in stopWords:
        if ( word == str ):
            response = False;    
    return response

annotations_dir = '../annotations/'
annotations_file_names = os.listdir(annotations_dir)
dataset_names = [x.split('_')[1] for x in annotations_file_names]
classes = {}
for dataset_name in dataset_names:
    target_name = "intermediate/purged_%s_tweets.txt" % dataset_name
    print 'Processing dataset %s...' % dataset_name
    target = open(target_name, 'w')
    classes[dataset_name] = []
    annotations_file = open('../annotations/annotations_%s_tweets.csv' % dataset_name,'r')
    for line in annotations_file:
        target.write(get_tweet(line) + '\n')
        classes[dataset_name].append(line.rstrip().split(';')[1:])
    target.close()
    command = ['"NLPToolkit.exe"','-pre', '-pos', '-lm', '"'+target_name+'"', '"'+target_name.replace('purged','pre_purged')+'"']
    #print ' '.join(command)
    call(' '.join(command),shell=True)
    
    target = open('processed/processed_%s_tweets.txt' % dataset_name, 'w')
    f = open('intermediate/pre_purged_%s_tweets.txt' % dataset_name,'r')
    counter = 0;
    for line in f:
        if ( line != "\n"):
            word = line.split('\t')[2].strip().lower()
            #if important(word):
            target.write(word+' ')
        else:
            #target.write('<|>'+classes[dataset_name][counter][0]+'<|>'+classes[dataset_name][counter][1]+'\n')
            target.write('<|>'+classes[dataset_name][counter][1]+'\n')
            counter = counter + 1
    f.close()
    target.close()
print 'Processing finished!'

Processing dataset bugonline...
Processing dataset hrtsport...
Processing dataset indexhr...
Processing dataset oglasnik...
Processing dataset politikaplus...
Processing dataset posaohr...
Processing finished!
